# U-Net Model 

In [11]:
# To connect google drive:     
from google.colab import drive   
drive.mount('/content/drive')    

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [13]:
 pip install segmentation-models      

     |████████████████████████████████| 51kB 5.4MB/s 


In [14]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()    

#initializer = tf.contrib.layers.xavier_initializer(seed=1) 
initializer = tf.truncated_normal_initializer(stddev=0.1)

Instructions for updating:
non-resource variables are not supported in the long term


In [15]:
import segmentation_models as sm

sm.set_framework('tf.keras')

sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [ ]:
#!pip install tensorflow-gpu==2.1.0    

In [ ]:
#!pip install keras==2.4.3

In [16]:
# Dependencies:
import tensorflow as tf 
import keras
import segmentation_models as sm    
import glob     
import cv2 
import os 
import numpy as np 
from matplotlib import pyplot as plt 

In [17]:
print(tf.__version__)  
print(keras.__version__)

2.5.0
2.5.0


In [18]:
BACKBONE = 'resnet101' 
preprocess_input = sm.get_preprocessing(BACKBONE)  

SIZE_X = 128 #Resize images (height  = X, width = Y) 
SIZE_Y = 128

In [20]:
# Path to Data:
images_path = "/content/drive/MyDrive/Data_set/Aug_Data/aug_img"
mask_path = "/content/drive/MyDrive/Data_set/Aug_Data/aug_masks" 
#images_path = "/content/drive/MyDrive/Grapes Model/JPEGImages"
#mask_path = "/content/drive/MyDrive/Grapes Model/SegmentationClassPNG" 

In [21]:
# Train_Data:
train_data = [] 
for img in sorted(os.listdir(images_path)):     
  image  = cv2.imread(os.path.join(images_path,img))  
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
  image = cv2.resize(image, (SIZE_X, SIZE_Y))  
  train_data.append(image) 

In [ ]:
Train_data = np.array(train_data) /255.0   
print("Train set Shape: ",Train_data.shape)      

In [ ]:
from google.colab.patches import cv2_imshow 

In [ ]:
cv2_imshow(cv2.resize(train_data[1],(256,256)))  

In [ ]:
# Mask_Data: 
mask_data = []
for img in sorted(os.listdir(mask_path)):  
  image  = cv2.imread(os.path.join(mask_path,img))   
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)     
  image = cv2.resize(image, (SIZE_X, SIZE_Y))  
  mask_data.append(image) 


In [ ]:
Mask_data = np.array(mask_data)  
Mask_data = Mask_data/255.0  
print("Mask set Shape: ",Mask_data.shape)   

In [ ]:
Mask_data = np.expand_dims(Mask_data, axis=3)  
print("Mask set Shape: ",Mask_data.shape)  

In [ ]:
cv2_imshow(cv2.resize(mask_data[1],(256,256)))    

In [ ]:
# Split Data
from sklearn.model_selection import train_test_split    
x_train, x_val, y_train, y_val = train_test_split(Train_data,Mask_data, test_size=0.2, random_state=42) 

In [ ]:
#print(x_train[0])

In [ ]:
Train_data = Train_data[0:421]

In [ ]:
# preprocess input
x_train = preprocess_input(x_train) 
x_val = preprocess_input(x_val)  

In [ ]:
#print(type(x_train[0]))

In [ ]:
# define model   
model = sm.Unet('resnet101', encoder_weights='imagenet') 

In [ ]:
model.compile(optimizer='adam',loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score])     

In [ ]:
print(model.summary())

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping 
#CallBacks = [ModelCheckpoint('/content/drive/MyDrive/Weights/Unet_Grapes_models.h5', monitor = 'val_loss', verbose=1, save_best_only = True, save_weights_only=False, mode='auto'),
#              EarlyStopping(monitor='val_loss', min_delta = 0, patience=30, verbose=1, mode='auto')]  

In [ ]:
import time, psutil
uptime = time.time() - psutil.boot_time()
remain = 12*60*60 - uptime
remain

In [2]:
history=model.fit(x_train,     
          y_train,    
          batch_size=32,   
          epochs=3000,        
          verbose=1,    
          validation_data=(x_val, y_val)) 

model.save('/content/drive/MyDrive/ Model/unet_tree_model.h5')    

NameError: ignored

In [ ]:
#model.save('/content/drive/MyDrive/ Model/unet_tree_model.h5')    

In [ ]:
x = 400 
#model.save('unet_model.h5')     

In [ ]:
#MODEL = keras.models.load_model('/content/drive/MyDrive/ Model/Unet_Grapes_models.h5')  

In [ ]:
MODEL = keras.models.load_model('/content/drive/MyDrive/ Model/unet_tree_model.h5',compile = False)  

In [ ]:
test_image_path = "/content/drive/MyDrive/Grapes_Detection/augmented_data/aug_img/img1.png"  
test_image = cv2.cvtColor(cv2.imread(test_image_path),cv2.COLOR_BGR2RGB)   
test_image = cv2.resize(test_image,(128,128))  

In [ ]:
#test_image =  x_val[100] 
#test_image = preprocess_input(test_image)  
   
cv2_imshow(cv2.resize(test_image,(256,256))) 

In [ ]:
Test_image = test_image/255.0 

In [ ]:
Test_image = np.expand_dims(Test_image,axis = 0)    
mask = MODEL.predict(Test_image)  

In [ ]:
#mask.shape 
mask = (mask[0]>0.01)*255  

In [ ]:
Mask = cv2.resize(np.array(mask).astype('float32'),(256,256))  
cv2_imshow(Mask)  

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 
cv2_imshow(Mask) 

In [ ]:
# IOU
iou_mask_real = cv2.imread("/content/drive/MyDrive/augmented_data/aug_mask/mask_1.png") 
iou_mask_real = cv2.cvtColor(iou_mask_real,cv2.COLOR_BGR2GRAY)
iou_mask_real = np.array(cv2.resize(iou_mask_real, (SIZE_X, SIZE_Y)))/255.0 
iou_mask_real = (iou_mask_real>0.01)*255    

In [ ]:
iou_mask_real = np.expand_dims(iou_mask_real, axis=2) 
iou_mask_real.shape 

In [ ]:
#IOU SCORE
intersection = np.logical_and(mask, iou_mask_real) 
union = np.logical_or(mask,iou_mask_real)
iou_score = np.sum(intersection) / np.sum(union) 
print("IOU SCORE: ",iou_score)

Another Test

In [ ]:
test_image_path_1 = "/content/drive/MyDrive/augmented_data/aug_img/img5.png"  
test_image_1 = cv2.cvtColor(cv2.imread(test_image_path_1),cv2.COLOR_BGR2RGB)   
test_image_1 = cv2.resize(test_image_1,(128,128))  
Test_image_1 = test_image_1/255.0  
Test_image_1 = np.expand_dims(Test_image_1,axis = 0)   

In [ ]:
cv2_imshow(cv2.resize(test_image_1,(256,256))) 

In [ ]:
pred_mask_1 = MODEL.predict(Test_image_1) 
pred_mask_1 = (pred_mask_1[0]>0.01)*255   

In [ ]:
cv2_imshow(cv2.resize(test_image_1,(256,256)))
cv2_imshow(cv2.resize(np.array(pred_mask_1).astype('float32'),(256,256)))   

In [ ]:
test_mask_1 = "/content/drive/MyDrive/augmented_data/aug_mask/mask_5.png" 
test_mask_1 = cv2.cvtColor(cv2.imread(test_mask_1),cv2.COLOR_BGR2RGB)   
test_mask_1 = cv2.resize(test_mask_1,(128,128))  
Test_mask_1 = np.array(test_mask_1)/255.0  
Test_mask_1 = (Test_mask_1>0.01)*255  

In [ ]:
#IOU SCORE 2
intersection = np.logical_and(pred_mask_1, Test_mask_1) 
union = np.logical_or(pred_mask_1, Test_mask_1)  
iou_score = np.sum(intersection) / np.sum(union)
print("IOU SCORE: ",iou_score)

In [ ]:
test_image_path = "/content/drive/MyDrive/Trunk_Detection/Dataset/Test/1.jpg"  
test_image = cv2.cvtColor(cv2.imread(test_image_path),cv2.COLOR_BGR2RGB)   
test_image = cv2.resize(test_image,(128,128)) 

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 

In [ ]:
Test_image = test_image/255.0 

In [ ]:
Test_image = np.expand_dims(Test_image,axis = 0)    
mask = MODEL.predict(Test_image)  

In [ ]:
#mask.shape 
mask = (mask[0]>0.01)*255  

In [ ]:
Mask = cv2.resize(np.array(mask).astype('float32'),(256,256))  
cv2_imshow(Mask)  

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 
cv2_imshow(Mask)

In [ ]:
test_image_path = "/content/drive/MyDrive/Trunk_Detection/Dataset/Test/2.jpg"  
test_image = cv2.cvtColor(cv2.imread(test_image_path),cv2.COLOR_BGR2RGB)   
test_image = cv2.resize(test_image,(128,128)) 

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 

In [ ]:
Test_image = test_image/255.0 

In [ ]:
Test_image = np.expand_dims(Test_image,axis = 0)    
mask = MODEL.predict(Test_image)  

In [ ]:
#mask.shape 
mask = (mask[0]>0.01)*255  

In [ ]:
Mask = cv2.resize(np.array(mask).astype('float32'),(256,256))  
cv2_imshow(Mask)  

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 
cv2_imshow(Mask)

In [ ]:
test_image_path = "/content/drive/MyDrive/Trunk_Detection/Dataset/Test/3.jpg"  
test_image = cv2.cvtColor(cv2.imread(test_image_path),cv2.COLOR_BGR2RGB)   
test_image = cv2.resize(test_image,(128,128)) 

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 

In [ ]:
Test_image = test_image/255.0 

In [ ]:
Test_image = np.expand_dims(Test_image,axis = 0)    
mask = MODEL.predict(Test_image)  

In [ ]:
#mask.shape 
mask = (mask[0]>0.01)*255  

In [ ]:
Mask = cv2.resize(np.array(mask).astype('float32'),(256,256))  
cv2_imshow(Mask)  

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 
cv2_imshow(Mask)

In [ ]:
test_image_path = "/content/drive/MyDrive/Trunk_Detection/Dataset/Test/4.jfif"  
test_image = cv2.cvtColor(cv2.imread(test_image_path),cv2.COLOR_BGR2RGB)   
test_image = cv2.resize(test_image,(128,128)) 

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 

In [ ]:
Test_image = test_image/255.0 

In [ ]:
Test_image = np.expand_dims(Test_image,axis = 0)    
mask = MODEL.predict(Test_image)  

In [ ]:
#mask.shape 
mask = (mask[0]>0.01)*255  

In [ ]:
Mask = cv2.resize(np.array(mask).astype('float32'),(256,256))  
cv2_imshow(Mask)  

In [ ]:
cv2_imshow(cv2.resize(test_image,(256,256))) 
cv2_imshow(Mask)